<h3> Capstone Project <h3>
   

<h4> Introduction/Business problem <h4>

The problem: Which London underground stations could be classified as residential, leisure or business?

Once categorised we would know what each station is being primKnowing the answer to this problem could help a start up business choose which area to open in or perhaps a town planner to understand the flow of the London population when considering how to further extend the rail system.

   

<h4> Data <h4>

The data used for this project will be a list of London undergrounds and their coordiantes from https://wiki.openstreetmap.org/wiki/List_of_London_Underground_stations. 

We will also need to know the venues surrounding each station (within a certain radius) and what type of venue it is to categorise the station. We can find these by using the Foursquare API which incidently comes with it's own set of categories that we will use.

    Arts & Entertainment
    College & University
    Event
    Food
    Nightlife Spot
    Outdoors & Recreation
    Professional & Other Places
    Residence
    Shop & Service
    Travel & Transport
    